In [1]:
#!git clone https://github.com/tGhattas/medical-images-utils.git


In [2]:
#!pip install scikit-image==0.18.1

In [2]:
!ls

__pycache__                    processor.py
final_liver_segmentation.ipynb


In [2]:
import os
from processor import *
from sys import platform
import nibabel as nib
import matplotlib.pyplot as plt
from skimage import data
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [3]:
%matplotlib notebook
# %matplotlib inline

In [4]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [5]:

from datetime import datetime as dt
def multi_slice_viewer(volumes):
    volumes = volumes if isinstance(volumes, list) else [volumes]
    figs = []
    for j in range(len(volumes)):

        fig, axes = plt.subplots(1, 3)
        volume = volumes[j]
        for i in range(3):
            ax = axes[i]
            ax.volume = volume
            ax.index = volume.shape[i] // 2
            where = [slice(-1,1,-1), slice(-1,1,-1), slice(-1,1,-1)]
            where[i] = ax.index
            where = tuple(where)
            ax.imshow(volume[where])
        
#     fig.canvas.mpl_connect('scroll_event', process_key)
#         fig.canvas.mpl_connect('key_press_event', process_key)
#         fig.canvas.draw()
        figs.append(fig)
    return figs if len(figs) > 1 else figs[0]


def select_slice(ax, ind, axis):
    axz = [ax] if not isinstance(ax, list) else ax
    for ax in axz:
        volume = ax.volume
        where = [slice(-1,1,-1), slice(-1,1,-1), slice(-1,1,-1)]
        print('---- ind ', ind)
        ax.index = int(ind) % volume.shape[axis]
        print('---- volume.shape ', volume.shape)
        where[axis] = ax.index
        print('---- ax.index ', ax.index)
        where = tuple(where)
        print('---- images len ', len(ax.images))
        print('---- where', where)
        ax.images[0].set_array(volume[where])
        
     
    
def process_key(event):
    fig = event.canvas.figure
    ax = fig.axes[0]
    if event.key == 'j':
        previous_slice(ax)
    elif event.key == 'k':
        next_slice(ax)
    fig.canvas.draw()

def previous_slice(ax):
    """Go to the previous slice."""
    volume = ax.volume
    ax.index = (ax.index - 1) % volume.shape[0]  # wrap around using %
    ax.images[0].set_array(volume[ax.index])

def next_slice(ax):
    """Go to the next slice."""
    volume = ax.volume
    ax.index = (ax.index + 1) % volume.shape[0]
    ax.images[0].set_array(volume[ax.index])
    


In [8]:

desc = ['Sagital', 'Coronal', 'Axial']
sliders_keeper = []
plot_keeper = []


In [8]:
import time
import numpy as np
import plotly.graph_objects as go

dir = './../data'
nifti_file = nib.load(os.path.join(dir, 'Case1_CT.nii'))
ct = nifti_file.get_fdata()

# ct = np.swapaxes(ct,0,2)

volume = ct.T
r, c = volume[0].shape

# Define frames

nb_frames = 68

fig = go.Figure(frames=[go.Frame(data=go.Surface(
    z=(6.7 - k * 0.1) * np.ones((r, c)),
    surfacecolor=np.flipud(volume[67 - k]),
    cmin=0, cmax=200
    ),
    name=str(k) # you need to name the frame for the animation to behave properly
    )
    for k in range(nb_frames)])

# Add data to be displayed before animation starts
fig.add_trace(go.Surface(
    z=6.7 * np.ones((r, c)),
    surfacecolor=np.flipud(volume[67]),
    colorscale='Gray',
    cmin=0, cmax=200,
    colorbar=dict(thickness=20, ticklen=4)
    ))


def frame_args(duration):
    return {
            "frame": {"duration": duration},
            "mode": "immediate",
            "fromcurrent": True,
            "transition": {"duration": duration, "easing": "linear"},
        }

sliders = [
            {
                "pad": {"b": 10, "t": 60},
                "len": 0.9,
                "x": 0.1,
                "y": 0,
                "steps": [
                    {
                        "args": [[f.name], frame_args(0)],
                        "label": str(k),
                        "method": "animate",
                    }
                    for k, f in enumerate(fig.frames)
                ],
            }
        ]

# Layout
fig.update_layout(
         title='Slices in volumetric data',
         width=600,
         height=600,
         scene=dict(
                    zaxis=dict(range=[-0.1, 6.8], autorange=False),
                    aspectratio=dict(x=1, y=1, z=1),
                    ),
         updatemenus = [
            {
                "buttons": [
                    {
                        "args": [None, frame_args(50)],
                        "label": "&#9654;", # play symbol
                        "method": "animate",
                    },
                    {
                        "args": [[None], frame_args(0)],
                        "label": "&#9724;", # pause symbol
                        "method": "animate",
                    },
                ],
                "direction": "left",
                "pad": {"r": 10, "t": 70},
                "type": "buttons",
                "x": 0.1,
                "y": 0,
            }
         ],
         sliders=sliders
)

fig.show()